IBM Applied Data Science Capstone 

Week 5 Final Report

Opening a New Supermarket in Amsterdam, the Netherlands

Build a dataframe of areas in Amsterdam by web scraping the data from Wikipedia page
Get the geographical coordinates of the areas
Obtain the venue data from Foursquare API
Explore and cluster the areas
Select the best cluster to open a new supermarket


1. Import libraries

In [2]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


2. Scrap data from Wikipedia page into a DataFrame

In [4]:
# sending the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighbourhoods_of_Amsterdam").text

In [5]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [6]:
# create a list to store neighborhood data
neighborhoodList = []

In [7]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [8]:
# create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

kl_df.head()

,Neighborhood
0,Template:Neighborhoods of Amsterdam
1,Admiralenbuurt
2,Amsteldorp
3,Amsterdam Oud-West
4,Amsterdam Oud-Zuid


In [9]:
# print the number of rows of the dataframe
kl_df.shape

(106, 1)

3. Get the geographical coordinates

In [10]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Amsterdam, Netherlands'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [11]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [12]:
coords

[[52.400720000000035, 4.9352700000000596],
 [52.372734366277776, 4.856363324463683],
 [52.36053999495108, 4.905160036643774],
 [52.36539000000005, 4.870220000000074],
 [52.35235000000006, 4.8778800000000615],
 [52.35430000000008, 4.958010000000058],
 [52.3502983469068, 4.868131315191405],
 [52.423407, 4.8333960000000165],
 [52.309891307795496, 4.9744197951663],
 [52.36993000000007, 4.907880000000034],
 [52.37919000000005, 4.8517400000000634],
 [52.408070000000066, 4.918960000000027],
 [52.396389977783066, 4.935390015580615],
 [52.33093000000008, 4.875180000000057],
 [52.30486817046341, 4.950105482979688],
 [52.3881068403939, 4.968097064532244],
 [52.37169000000006, 4.8972400000000675],
 [52.36918565463759, 4.861479786899981],
 [52.36993000000007, 4.907880000000034],
 [52.369760019908455, 4.928991749234822],
 [52.362774487416154, 4.927368239783042],
 [52.352813911885875, 4.7767431053451],
 [52.356250000000045, 4.890570000000025],
 [52.37169000000006, 4.8972400000000675],
 [52.3511236479

In [13]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [14]:
# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

In [15]:
# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df

(106, 3)


,Neighborhood,Latitude,Longitude
0,Template:Neighborhoods of Amsterdam,52.400720,4.935270
1,Admiralenbuurt,52.372734,4.856363
2,Amsteldorp,52.360540,4.905160
3,Amsterdam Oud-West,52.365390,4.870220
4,Amsterdam Oud-Zuid,52.352350,4.877880
5,Amsterdam Science Park,52.354300,4.958010
6,Apollobuurt,52.350298,4.868131
7,Betondorp,52.423407,4.833396
8,Bijlmermeer,52.309891,4.974420
9,Binnenstad (Amsterdam),52.369930,4.907880


In [16]:
# save the DataFrame as CSV file
kl_df.to_csv("kl_df.csv", index=False)

4. Create a map of Amsterdam with neighborhoods superimposed on top

In [17]:
# get the coordinates of Amsterdam
address = 'Amsterdam, the Netherlands'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Amsterdam, the Netherlands {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Amsterdam, the Netherlands 52.3727598, 4.8936041.


In [18]:
# create map of Amsterdam using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [19]:
# save the map as HTML file
map_kl.save('map_kl.html')

5. Use the Foursquare API to explore the neighborhoods

In [20]:
# define Foursquare Credentials and Version
CLIENT_ID = 'I34UIWOCA1TLGZ5WQ1HIS5OKRBNLNPGC3U45W4KFRS2PRJBK' # your Foursquare ID
CLIENT_SECRET = '21BJAHSGEMVQLZ3AWB414OVLRWZGZSDCOGW0IABI5NAIYT2C' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: I34UIWOCA1TLGZ5WQ1HIS5OKRBNLNPGC3U45W4KFRS2PRJBK
CLIENT_SECRET:21BJAHSGEMVQLZ3AWB414OVLRWZGZSDCOGW0IABI5NAIYT2C


Now, let's get the top 100 venues that are within a radius of 2000 meters.

In [21]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [22]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(9435, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Template:Neighborhoods of Amsterdam,52.40072,4.93527,Musashi,52.398241,4.936290,Japanese Restaurant
1,Template:Neighborhoods of Amsterdam,52.40072,4.93527,Sportcentrum Sport-Natural,52.401544,4.931235,Gym / Fitness Center
2,Template:Neighborhoods of Amsterdam,52.40072,4.93527,Patisserie Kwekkeboom BovenIJ,52.397824,4.937422,Bakery
3,Template:Neighborhoods of Amsterdam,52.40072,4.93527,Café 't Sluisje,52.391134,4.938363,Bar
4,Template:Neighborhoods of Amsterdam,52.40072,4.93527,Noorderparkbad,52.398171,4.918952,Pool


In [23]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Japanese Restaurant', 'Gym / Fitness Center', 'Bakery', 'Bar',
       'Pool', 'Ice Cream Shop', 'Café', 'Camera Store', 'Golf Course',
       'Park', 'Mexican Restaurant', 'Skate Park', 'Church', 'Rock Club',
       'French Restaurant', 'Supermarket', 'Campground', 'Climbing Gym',
       'Grocery Store', 'Indie Movie Theater', 'Turkish Restaurant',
       'Athletics & Sports', 'Caribbean Restaurant', 'Discount Store',
       'Ethiopian Restaurant', 'Brewery', 'Thai Restaurant', 'Restaurant',
       'Gym', 'Gastropub', 'Sandwich Place', 'Market', 'Drugstore',
       'Clothing Store', 'Plaza', 'Soccer Field', 'Indonesian Restaurant',
       'Electronics Store', 'Asian Restaurant', 'Shopping Mall',
       'Seafood Restaurant', 'Hotel', 'Department Store', 'Pharmacy',
       'Pool Hall', 'Jewelry Store', 'Mobile Phone Shop', 'Music Venue',
       'Bus Stop', 'Snack Place'], dtype=object)

In [24]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True


6. Analyze Each Neighborhood

In [25]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(9435, 292)


,Neighborhoods,Accessories Store,Adult Boutique,Advertising Agency,African Restaurant,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Bath House,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Boarding House,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Brasserie,Breakfast Spot,Brewery,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Campground,Canal,Canal Lock,Candy Store,Caribbean Restaurant,Casino,Castle,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cruise,Cultural Center,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Doner Restaurant,Drugstore,Dutch Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Friterie,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Heliport,Hill,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music School,Music Venue,Neighborhood,Nightclub,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoor Supply Store,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Perfume Shop,Pet Café,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Resort,Rest Area,Restaurant,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Repair,Shopping Mall,Skate Park,Skating Rink,Ski Area,Ski Chairlift,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Spiritual Center,Sporting Goods Shop,Sports Club,Squash Court,Stables,Stadium,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tour Provider,Toy / Game Store,Track,Trail,Train Station,Tram Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Water Park,Whisky Bar,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Template

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [26]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(106, 292)


,Neighborhoods,Accessories Store,Adult Boutique,Advertising Agency,African Restaurant,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Bath House,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Boarding House,Boat or Ferry,Bookstore,Border Crossing,Botanical Garden,Boutique,Bowling Alley,Brasserie,Breakfast Spot,Brewery,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Campground,Canal,Canal Lock,Candy Store,Caribbean Restaurant,Casino,Castle,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cruise,Cultural Center,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Doner Restaurant,Drugstore,Dutch Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Friterie,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Heliport,Hill,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music School,Music Venue,Neighborhood,Nightclub,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoor Supply Store,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Perfume Shop,Pet Café,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Print Shop,Pub,Public Art,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Resort,Rest Area,Restaurant,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Repair,Shopping Mall,Skate Park,Skating Rink,Ski Area,Ski Chairlift,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Spiritual Center,Sporting Goods Shop,Sports Club,Squash Court,Stables,Stadium,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tour Provider,Toy / Game Store,Track,Trail,Train Station,Tram Station,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Water Park,Whisky Bar,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Admirale

In [28]:
len(kl_grouped[kl_grouped["Supermarket"] > 0])

50

Create a new DataFrame for supermarket data only

In [29]:
kl_supermarket = kl_grouped[["Neighborhoods","Supermarket"]]

In [30]:

kl_supermarket.head()

,Neighborhoods,Supermarket
0,Admiralenbuurt,0.00
1,Amsteldorp,0.00
2,Amsterdam Oud-West,0.00
3,Amsterdam Oud-Zuid,0.00
4,Amsterdam Science Park,0.02



7. Cluster Neighborhoods

Run k-means to cluster the neighborhoods in Amsterdam into 3 clusters.

In [32]:
# set number of clusters
kclusters = 3

kl_clustering = kl_supermarket.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 2, 0, 0])

In [33]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_supermarket.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [34]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Supermarket,Cluster Labels
0,Admiralenbuurt,0.00,0
1,Amsteldorp,0.00,0
2,Amsterdam Oud-West,0.00,0
3,Amsterdam Oud-Zuid,0.00,0
4,Amsterdam Science Park,0.02,0


In [35]:
# merge Amsterdam_grouped with Amsterdam_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(106, 5)


,Neighborhood,Supermarket,Cluster Labels,Latitude,Longitude
0,Admiralenbuurt,0.00,0,52.372734,4.856363
1,Amsteldorp,0.00,0,52.360540,4.905160
2,Amsterdam Oud-West,0.00,0,52.365390,4.870220
3,Amsterdam Oud-Zuid,0.00,0,52.352350,4.877880
4,Amsterdam Science Park,0.02,0,52.354300,4.958010


In [36]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(106, 5)


,Neighborhood,Supermarket,Cluster Labels,Latitude,Longitude
0,Admiralenbuurt,0.000000,0,52.372734,4.856363
74,Prinses Irenebuurt,0.010000,0,52.341769,4.875224
73,Plantage,0.000000,0,52.366830,4.912160
71,Overtoombuurt,0.000000,0,52.359210,4.861250
70,Overhoeks,0.010000,0,52.391680,4.905780
69,Oude Pijp,0.000000,0,52.356250,4.890570
68,Oud-Oost,0.000000,0,52.360130,4.925320
64,Oostpoort,0.000000,0,52.359319,4.903081
62,Oosterparkbuurt (Amsterdam),0.000000,0,52.359470,4.922980
61,Oosterdokseiland,0.000000,0,52.369930,4.907880


Finally, let's visualize the resulting clusters

In [37]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [38]:
# save the map as HTML file
map_clusters.save('map_clusters.html')


8. Examine Clusters